# Imports & Settings


In [2]:
# Imports
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from deep_translator import GoogleTranslator
import re 
from math import isnan

pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_seq_items', None)

In [10]:
# dtype={'type': str} prevents being confused with data type for large data sets
train = pd.read_csv('data/train.csv', index_col='id', dtype={'type': str})
test = pd.read_csv('data/test.csv', index_col='id', dtype={'type': str})
train_translated = pd.read_csv('data/train_translated.csv', dtype={'type': str})
test_translated = pd.read_csv('data/test_translated.csv', index_col='id', dtype={'type': str})
combined_data = pd.read_csv('data/combined_data.csv', index_col='id', dtype={'type': str})
combined_data_translated = pd.read_csv('data/combined_data_translated.csv', index_col='id', dtype={'type': str})
combined_data_fully_translated = pd.read_csv('data/combined_data_fully_translated.csv', index_col='id', dtype={'type': str})
prep = pd.read_csv('data/prep.csv', index_col='id', dtype={'type': str})

/tmp/ipykernel_21062/1699356385.py:9: DtypeWarning: Columns (11,264) have mixed types. Specify dtype option on import or set low_memory=False.
  prep = pd.read_csv('data/prep.csv', index_col='id', dtype={'type': str})


# Delete unneeded features


In [11]:
data = prep.copy()

In [14]:
data.head(1)

,ks,event_type,location,before_Christ,participants_role,participant,parish,class,museum_abbr,musealia_mark,musealia_seria_nr,musealia_queue_nr,musealia_additional_nr,collection_mark,collection_queue_nr,collection_additional_nr,element_count,is_original,damages,state,color,type,source,city_municipality,country,material_Polish,material_RC Photo Paper,material_a pearl,material_acetate cellulose film,material_albumen paper,material_albumin paper,material_aluminium,material_amber,material_artificial fiber material,material_artificial leather,material_artificial material,material_atlas,material_ballpoint pen ink,material_birch,material_bone,material_brass,material_brocade (clothing variety),material_bronze,material_canvas,material_canvas (type of cloth),material_cardboard,material_cast iron,material_celluloid,material_ceramics,material_chalk,material_chamois leather,material_chamotte,material_charcoal,material_chromogen emulsion,material_chromogen paper,material_clay,material_clothing variety,material_collodion paper,material_colloid paper,material_colour,material_copper,material_cotton,material_crepe,material_crepe (cloth type),material_crystal,material_diffusion paper,material_email,material_emulsion,material_enamel paint,material_faience,material_feather,material_film,material_film (material),material_flint,material_from the bat,material_glass,material_gold,material_granite,material_graphite,material_gypsum,material_handmade paper,material_ink,material_iron,material_kalka,material_knitwear,material_leotard (type of clothing),material_linen,material_mascara,material_metal,material_metal fibers,material_moire (clothing variety),material_movie,material_nan,material_newsprint,material_nitrocellulose film,material_nut,material_oil paint,material_organic matter,material_paper,material_papier mache,material_photo emulsion,material_photo material,material_photo paper,material_photo plate,material_photographic material,material_plastic,material_plastic mass,material_plywood,material_porcelain,material_printing ink,material_quartz,material_rubber,material_salt paper,material_silk,material_silver,material_silver gelatin emulsion,material_silver gelatin paper,material_skin,material_slate,material_stone,material_synthetic fibers,material_synthetic material,material_tempera,material_textile,material_tin,material_trillion,material_watercolor paint,material_wax,material_white metal,material_wire,material_wood,material_wood material,material_wooden board,material_wool,material_yarn,technique_(close/together) sewing,technique_addition,technique_aquatint,technique_ballpoint pen,technique_binding techniques,technique_black and white photo,technique_black and white photography,technique_bronzing,technique_brushing,technique_burning,technique_carving,technique_casting,technique_chalk,technique_charcoal,technique_chromogen procedure,technique_collage,technique_collotype,technique_color photo,technique_color photography,technique_colored chalk,technique_colored pencil,technique_coloring,technique_copper engraving,technique_copying,technique_crayon drawing,technique_croaking,technique_crocheting,technique_cutting,technique_cuttlefish,technique_daguerreotype,technique_digital photography,technique_digital printing,technique_drawing,technique_electronic imaging,technique_embroidery,technique_enamelling,technique_engraving,technique_etching,technique_felt tip pen,technique_glass technology,technique_gouache,technique_graphics,technique_graphite,technique_handicraft,technique_handwriting,technique_ink,technique_ink drawing,technique_intarsia,technique_kiln ceramics,technique_knitting,technique_letterpress with raster cliché,technique_linocut,technique_lithography,technique_manuscript,technique_marker,technique_mascara,technique_mezzotinto,technique_mixed media,technique_modeling,technique_molding,technique_monotypy,technique_nan,technique_offset printing,technique_oil,technique_oil painting,technique_painting,technique_painting techniques,technique_pannotype

In [13]:
data.drop(columns=['full_nr', 'name','commentary','country_and_unit','text','parameter','unit','value','legend','initial_info','additional_text'], inplace=True)

In [15]:
data.to_csv('data/prep.csv')